### Purpose: Clean and transform raw FDS output into an eVestment Alliance friendly upload file

In [71]:
# Import dependencies
import numpy as np
import pandas as pd
import xlsxwriter

#### Step 1 - Market Cap

In [72]:
# Ensure all mkt cap ranges are preserved even if not in FDS report (because no weight in a particular period)
buckets = pd.read_csv('buckets.csv')

In [74]:
# Read in 'mc' tab from excel file containing the FDS market cap output
mc_df = pd.read_excel('eA_template.xlsx', sheet_name = 'mc', skiprows=5, skipfooter =2) 
headers = mc_df.iloc[0]
mc_df = pd.DataFrame(mc_df.values[2:], columns=headers)
mc_df.columns.values[0] = 'Label'
mc_df

,Label,ABC,XYX,ZZZ
0,> $50 Billion,43.9,84.52,37.63
1,$25-50 Billion,32.46,14.49,18.93
2,$15-25 Billion,11.972483,0.989023,15.315204
3,$10-15 Billion,7.205876,--,8.140186
4,$7.5-10 Billion,3.131328,--,5.43
5,$7-7.5 Billion,0.338347,--,3.61
6,$5-7 Billion,0.764615,--,6.655106
7,$2.5-5 Billion,0.228641,--,4.294145


In [75]:
# Merge mc_df and buckets to make sure have all the market cap ranges
mc = pd.merge(left = buckets, right = mc_df, how = 'left', on = "Label")

In [76]:
# get report data and report currency from mc tab
date_df = pd.read_excel('eA_template.xlsx', sheet_name = 'mc')
report_date = date_df.iloc[4,0]
report_currency = date_df.iloc[0,0]
print('Date: ', report_date)
print('Currency: ', report_currency)

Date:  3/31/2021
Currency:  USD


#### Step 2 - Transform market cap dataframe data into eVestment format

In [77]:
## Create list of all products included in MC USD tab.  Will provide as variable below.  This is scalable, if add new products.
mc_cols = list(mc.columns)
mc_cols = mc_cols[1:]
mc_cols

['ABC', 'XYX', 'ZZZ']

In [78]:
##  Repeat MC buckets and create Product Name column for each product
mc_melted = pd.melt(mc, id_vars=['Label'], value_vars = mc_cols,
var_name = 'Product', value_name = 'Value')

In [79]:
## Replace any '--' and 'na's' with 0
mc_melted['Value'].replace('--',0, inplace = True)
mc_melted['Value'].fillna(0, inplace = True)

In [80]:
## Add Date Column
mc_melted.insert(0, 'Date', report_date)

In [81]:
## Use 'Label' column to create 'eA Characteristic' column, this is only for market cap

def assign_mc_name(label):
    if label == '> $50 Billion':
        return 'Capitalization Ranges-> $50 Billion'
    elif label == '$25-50 Billion':
        return 'Capitalization Ranges-$25-50 Billion'
    elif label == '$15-25 Billion':
        return 'Capitalization Ranges-$15-25 Billion'
    elif label == '$10-15 Billion':
        return 'Capitalization Ranges-$10-15 Billion'
    elif label == '$7.5-10 Billion':
        return 'Capitalization Ranges-$7.5-10 Billion'
    elif label == '$7-7.5 Billion':
        return 'Capitalization Ranges-$7-7.5 Billion'
    elif label == '$5-7 Billion':
        return 'Capitalization Ranges-$5-7 Billion'
    elif label == '$2.5-5 Billion':
        return 'Capitalization Ranges-$2.5-5 Billion'
    elif label == '$2-2.5 Billion':
        return 'Capitalization Ranges-$2-2.5 Billion'
    elif label == '$1.5-2 Billion':
        return 'Capitalization Ranges-$1.5-2 Billion'
    elif label == '$750-1.5 Billion':
        return 'Capitalization Ranges-$750-1.5 Billion'
    elif label == '$500-750 Million':
        return 'Capitalization Ranges-$500-750 Million'
    elif label == '$400-500 Million':
        return 'Capitalization Ranges-$400-500 Million'
    elif label == '< $400 Million':
        return 'Capitalization Ranges-< $400 Million'
    else:
        return 'Houston, we have a prob'    

In [82]:
mc_melted['eA Characteristic'] = mc_melted['Label'].apply(assign_mc_name)

In [83]:
# Use 'Label' column to create 'Category' column, this is again, only for market cap

def assign_category(label):
    group = ('> $50 Billion', '$25-50 Billion', '$15-25 Billion', '$10-15 Billion', 
             '$7.5-10 Billion', '$7-7.5 Billion', '$5-7 Billion', '$2.5-5 Billion', 
             '$2-2.5 Billion', '$1.5-2 Billion', '$750-1.5 Billion', '$500-750 Million', 
             '$400-500 Million', '< $400 Million')
    
    if label in group:
        return 'Capitalization Ranges'
    else:
        return 'Houston, we have a prob'
    
# Apply the assign category function
mc_melted['Category'] = mc_melted['Label'].apply(assign_category)

# Rearrange order of cols
mc_melted = mc_melted[['Date','Product', 'Category', 'Label', 'Value', 'eA Characteristic']]

In [84]:
# Create a function to change product ID to eVestment Name

def change_name(product):
    if product == 'IEV-US':
        return 'iShares Europe ETF'
    elif product ==  'IOO-US':
        return 'ishares Global 100 ETF'
    elif product == 'EWJ-US':
        return 'ishares MSCI Japan ETF'
    else:
        return product

# Run the change_name function
mc_melted['Product'] = mc_melted['Product'].apply(change_name)

In [85]:
# Final market cap dataframe
mc_melted.head(10)

,Date,Product,Category,Label,Value,eA Characteristic
0,3/31/2021,ABC,Capitalization Ranges,> $50 Billion,43.900000,Capitalization Ranges-> $50 Billion
1,3/31/2021,ABC,Capitalization Ranges,$25-50 Billion,32.460000,Capitalization Ranges-$25-50 Billion
2,3/31/2021,ABC,Capitalization Ranges,$15-25 Billion,11.972483,Capitalization Ranges-$15-25 Billion
3,3/31/2021,ABC,Capitalization Ranges,$10-15 Billion,7.205876,Capitalization Ranges-$10-15 Billion
4,3/31/2021,ABC,Capitalization Ranges,$7.5-10 Billion,3.131328,Capitalization Ranges-$7.5-10 Billion
5,3/31/2021,ABC,Capitalization Ranges,$7-7.5 Billion,0.338347,Capitalization Ranges-$7-7.5 Billion
6,3/31/2021,ABC,Capitalization Ranges,$5-7 Billion,0.764615,Capitalization Ranges-$5-7 Billion
7,3/31/2021,ABC,Capitalization Ranges,$2.5-5 Billion,0.228641,Capitalization Ranges-$2.5-5 Billion
8,3/31/2021,ABC,Capitalization Ranges,$2-2.5 Billion,0.000000,Capitalization Ranges-$2-2.5 Billion
9,3/31/2021,ABC,Capitalization Ranges,$1.5-2 Billion,0.000000,Capitalization Ranges-$1.5-2 Billion


#### Step 3 - Import `Sector`  Tab

In [86]:
sector_df = pd.read_excel('eA_template.xlsx', sheet_name = 'sector',skiprows=5, skipfooter =2) 
sector_headers = sector_df.iloc[0]
sector_df = pd.DataFrame(sector_df.values[2:], columns=sector_headers)
sector_df.columns.values[0] = 'Label'
sector_df

,Label,ABC,XYX,ZZZ
0,Communication Services,5.73,6.45,7.63
1,Consumer Discretionary,9.86,16.48,20.71
2,Consumer Staples,12.449602,11.192407,7.072424
3,Energy,4.530575,4.817289,0.639922
4,Financials,16.423228,10.750525,9.062188
5,Health Care,13.460141,12.487556,10.008099
6,Industrials,15.278096,6.14538,21.071196
7,Information Technology,8.108733,28.162992,13.673895
8,Materials,8.302382,2.134073,5.231248
9,Real Estate,1.313005,0.63731,3.60585


#### Step 4 - Transform `sector` data into Seismic format

In [87]:
## Create list of all products included in sector tab.  Will provide as variable below.  This is scalable, if add new products.
sector_cols = list(sector_df.columns)
## Excludes Label and MSCI ACWI from list
sector_cols = sector_cols[1:]
sector_cols

['ABC', 'XYX', 'ZZZ']

In [88]:
##  Repeat Sector buckets and create Product Name column for each product
sector_melted = pd.melt(sector_df, id_vars=['Label'], value_vars = sector_cols,
var_name = 'Product', value_name = 'Value')

In [89]:
# Replace  -- from import file with 0
sector_melted['Value'].replace('--', 0, inplace = True)

In [90]:
## Removing Unassigned rows where 'Value' = 0
sector_melted = sector_melted.drop(sector_melted[(sector_melted['Label'] == '[Unassigned]') & (sector_melted['Value'] == 0.0)].index)

In [91]:
## Add Date Column
sector_melted.insert(0, 'Date', report_date)

In [92]:
## Create function to leverage sector names from 'Label' column to create relevant 'eA Characteristic' column

def assign_sector_name(label):
    if label == 'Communication Services':
        return 'Equity sector allocations-Communication Services'
    elif label == 'Consumer Discretionary':
        return 'Equity sector allocations-Consumer Discretionary'
    elif label == 'Consumer Staples':
        return 'Equity sector allocations-Consumer Staples'
    elif label == 'Energy':
        return 'Equity sector allocations-Energy'
    elif label == 'Financials':
        return 'Equity sector allocations-Financials'
    elif label == 'Health Care':
        return 'Equity sector allocations-Health Care'
    elif label == 'Industrials':
        return 'Equity sector allocations-Industrials'
    elif label == 'Information Technology':
        return 'Equity sector allocations-Information Technology'
    elif label == 'Materials':
        return 'Equity sector allocations-Materials'
    elif label == 'Real Estate':
        return 'Equity sector allocations-Real Estate'
    elif label == 'Utilities':
        return 'Equity sector allocations-Utilities'
    elif label == 'Telecommunication Services':
        return 'Equity sector allocations-Telecommunication Services' 
    else:
        return 'Unassigned'

In [93]:
# Run assign sector name function to create eA Characteristic column
sector_melted['eA Characteristic'] = sector_melted['Label'].apply(assign_sector_name)

In [94]:
sector_values = sector_melted['Label'].value_counts().index.to_list()
sector_values

['Materials',
 'Information Technology',
 'Energy',
 'Industrials',
 'Utilities',
 'Health Care',
 'Communication Services',
 'Real Estate',
 'Consumer Discretionary',
 'Financials',
 'Consumer Staples']

In [95]:
## Use 'Label' column to create 'Category' column, this is again, only for market cap

def assign_sector_category(label):
    if label in sector_values:
        return 'Equity sector allocations'
    else:
        return 'Houston, we have a prob'

In [96]:
sector_melted['Category'] = sector_melted['Label'].apply(assign_sector_category)

In [97]:
## Rearrange order of cols
sector_melted = sector_melted[['Date','Product', 'Category', 'Label', 'Value', 'eA Characteristic']]

In [98]:
## Run function to change eVestment Product name
sector_melted['Product'] = sector_melted['Product'].apply(change_name)
sector_melted.head(3)

,Date,Product,Category,Label,Value,eA Characteristic
0,3/31/2021,ABC,Equity sector allocations,Communication Services,5.73,Equity sector allocations-Communication Services
1,3/31/2021,ABC,Equity sector allocations,Consumer Discretionary,9.86,Equity sector allocations-Consumer Discretionary
2,3/31/2021,ABC,Equity sector allocations,Consumer Staples,12.449602,Equity sector allocations-Consumer Staples


#### Step 5 - Import `cash` Tab

In [99]:
cash_df = pd.read_excel('eA_template.xlsx', sheet_name = 'cash', skiprows=5, skipfooter =2) 
cash_headers = cash_df.iloc[0]
cash_df = pd.DataFrame(cash_df.values[2:], columns=headers)
cash_df.columns.values[0] = 'Label'

In [100]:
# Replace  -- from import file with 0
cash_df.replace('--', 0, inplace = True)

#### Step 6 - Transform `cash` data into Seismic format

In [101]:
## Create list of all products included in cash tab.  Will provide as variable below.  This is scalable, if add new products.
cash_cols = list(cash_df.columns)
cash_cols = cash_cols[1:]

In [102]:
## Create formula to assign US to Domestic and others to Foreign
def assign_dom_foreign(label):
    if label == 'United States':
        return 'Common Stock - Domestic'
    if label == '[Cash]':
        return 'Cash and Equivalents'
    else:
        return 'Common Stock - Foreign'

In [103]:
cash_df['Label'] = cash_df['Label'].apply(assign_dom_foreign)

In [104]:
cash_grouped = cash_df.groupby('Label')

In [105]:
cash_sums = cash_grouped.sum().reset_index()
cash_sums.head(10)

,Label,ABC,XYX,ZZZ
0,Cash and Equivalents,0.336563,0.109899,1.8
1,Common Stock - Domestic,0.000000,69.020000,0.0
2,Common Stock - Foreign,99.659494,30.865698,98.2


In [106]:
##  Repeat cash buckets and create Product Name column for each product
cash_melted = pd.melt(cash_sums, id_vars=['Label'], value_vars = cash_cols, var_name = 'Product', value_name = 'Value')

In [107]:
# Replace  -- from import file with 0
cash_melted['Value'].replace('--', 0, inplace = True)

In [108]:
## Removing Unassigned rows where 'Value' = 0
cash_melted = cash_melted.drop(cash_melted[(cash_melted['Label'] == '[Unassigned]') & (cash_melted['Value'] == 0.0)].index)

In [109]:
## Add Date Column
cash_melted.insert(0, 'Date', report_date)
cash_melted.head(3)

,Date,Label,Product,Value
0,3/31/2021,Cash and Equivalents,ABC,0.336563
1,3/31/2021,Common Stock - Domestic,ABC,0.000000
2,3/31/2021,Common Stock - Foreign,ABC,99.659494


In [110]:
## Use 'Label' column to create 'eA Characteristic' column

def assign_cash_name(label):
    if label == 'Common Stock - Domestic':
        return 'Asset Allocation-Common Stock - Domestic'
    elif label == 'Cash and Equivalents':
        return 'Asset Allocation-Cash and Equivalents'
    elif label == 'Common Stock - Foreign':
        return 'Asset Allocation-Common Stock - Foreign'    

In [111]:
cash_melted['eA Characteristic'] = cash_melted['Label'].apply(assign_cash_name)
cash_melted.head(5)

,Date,Label,Product,Value,eA Characteristic
0,3/31/2021,Cash and Equivalents,ABC,0.336563,Asset Allocation-Cash and Equivalents
1,3/31/2021,Common Stock - Domestic,ABC,0.000000,Asset Allocation-Common Stock - Domestic
2,3/31/2021,Common Stock - Foreign,ABC,99.659494,Asset Allocation-Common Stock - Foreign
3,3/31/2021,Cash and Equivalents,XYX,0.109899,Asset Allocation-Cash and Equivalents
4,3/31/2021,Common Stock - Domestic,XYX,69.020000,Asset Allocation-Common Stock - Domestic


In [112]:
cash_values = cash_melted['Label'].value_counts().index.to_list()
cash_values

['Cash and Equivalents', 'Common Stock - Domestic', 'Common Stock - Foreign']

In [113]:
## Use 'Label' column to create 'Category' column, this is again, only for market cap

def assign_cash_category(label):
    if label in cash_values:
        return 'Asset Allocation'
    else:
        return 'Houston, we have a prob'

In [114]:
# Run the eVestment Product name change function
cash_melted['Category'] = cash_melted['Label'].apply(assign_cash_category)

In [115]:
# Run function to change eVestment Product name
cash_melted['Product'] = cash_melted['Product'].apply(change_name)

In [116]:
## Rearrange order of cols
cash_melted = cash_melted[['Date','Product', 'Category', 'Label', 'Value', 'eA Characteristic']]
cash_melted.head(12)

,Date,Product,Category,Label,Value,eA Characteristic
0,3/31/2021,ABC,Asset Allocation,Cash and Equivalents,0.336563,Asset Allocation-Cash and Equivalents
1,3/31/2021,ABC,Asset Allocation,Common Stock - Domestic,0.000000,Asset Allocation-Common Stock - Domestic
2,3/31/2021,ABC,Asset Allocation,Common Stock - Foreign,99.659494,Asset Allocation-Common Stock - Foreign
3,3/31/2021,XYX,Asset Allocation,Cash and Equivalents,0.109899,Asset Allocation-Cash and Equivalents
4,3/31/2021,XYX,Asset Allocation,Common Stock - Domestic,69.020000,Asset Allocation-Common Stock - Domestic
5,3/31/2021,XYX,Asset Allocation,Common Stock - Foreign,30.865698,Asset Allocation-Common Stock - Foreign
6,3/31/2021,ZZZ,Asset Allocation,Cash and Equivalents,1.800000,Asset Allocation-Cash and Equivalents
7,3/31/2021,ZZZ,Asset Allocation,Common Stock - Domestic,0.000000,Asset Allocation-Common Stock - Domestic
8,3/31/2021,ZZZ,Asset Allocation,Common Stock - Foreign,98.200000,Asset Allocation-Common Stock - Foreign


#### Step 7 Import `chars` Tab

In [117]:
chars_df = pd.read_excel('eA_template.xlsx', sheet_name = 'chars', skiprows=5) 
chars_headers = chars_df.iloc[0]
chars_df = pd.DataFrame(chars_df.values[2:], columns = chars_headers)
chars_df.columns.values[0] = 'Label'
chars_df

,Label,ABC,XYX,ZZZ
0,Total Number of Stocks,422,159,640
1,Price/Earnings Ratio (1 Year Trailing),34.44,25.22,28.85
2,P/E Ratio (1Yr Forward),24.35,24.48,26.58
3,Dividend Yield,2.575805,2.339417,1.863503
4,Dividend Growth 1yr,-15.031019,0.936817,0.816302
5,Growth Rate (5 Year Trailing),4.505492,12.52202,6.028457
6,Growth Rate (1 Year Forward),23.023137,24.309815,5.948094
7,Growth Rate (3 - 5 Year Forward),10.874606,12.692258,12.071347
8,ROA,4.392247,8.984804,4.394433
9,ROE,11.689371,19.359811,8.448504


#### Step 8 - Transform `chars` data into Seismic format

In [118]:
## Create list of all products included in sector tab.  Will provide as variable below.  This is scalable, if add new products.
chars_cols = list(chars_df.columns)
chars_cols = chars_cols[1:]

In [119]:
##  Repeat Sector buckets and create Product Name column for each product
chars_melted = pd.melt(chars_df, id_vars=['Label'], value_vars = chars_cols, var_name = 'Product', value_name = 'Value')

In [120]:
## Add Date Column
chars_melted.insert(0, 'Date', report_date)

In [121]:
## Create function to leverage sector names from 'Label' column to create relevant 'eA Characteristic' column

def assign_chars_name(label):
    if label == 'Total Number of Stocks':
        return 'Equity characteristics-Total Number of Stocks'
    elif label == 'Price/Earnings Ratio (1 Year Trailing)':
        return 'Equity characteristics-Price/Earnings Ratio (1 Year Trailing)'
    elif label == 'P/E Ratio (1Yr Forward)':
        return 'Equity characteristics-P/E Ratio (1Yr Forward)'
    elif label == 'Dividend Yield':
        return 'Equity characteristics-Dividend Yield'
    elif label == 'Dividend Growth 1yr':
        return 'Equity characteristics-Dividend Growth 1yr'
    elif label == 'Growth Rate (5 Year Trailing)':
        return 'Equity characteristics-Growth Rate (5 Year Trailing)'
    elif label == 'Growth Rate (1 Year Forward)':
        return 'Equity characteristics-Growth Rate (1 Year Forward)'
    elif label == 'Growth Rate (3 - 5 Year Forward)':
        return 'Equity characteristics-Growth Rate (3 - 5 Year Forward)'
    elif label == 'ROA':
        return 'Equity characteristics-ROA'
    elif label == 'ROE':
        return 'Equity characteristics-ROE'
    elif label == 'ROE 5yr Avg':
        return 'Equity characteristics-ROE 5yr Avg'
    elif label == 'Price/Book Ratio':
        return 'Equity characteristics-Price/Book Ratio' 
    elif label == 'Price/Cash Flow':
        return 'Equity characteristics-Price/Cash Flow' 
    elif label == 'Price to Cash Flow 5yr Avg':
        return 'Equity characteristics-Price to Cash Flow 5yr Avg' 
    elif label == 'Price/Sales':
        return 'Equity characteristics-Price Sales' 
    elif label == 'Total Debt/Total Equity':
        return 'Equity characteristics-Total Debt/Total Equity' 
    elif label == 'Weighted Average Market Cap ($ millions)':
        return 'Equity characteristics-Weighted Average Market Cap ($ millions)' 
    elif label == 'Median Market Cap':
        return 'Equity characteristics-Median Market Cap' 
    elif label == 'Weighted Median Market Cap ($ millions)':
        return 'Equity characteristics-Weighted Median Market Cap ($ millions)' 
    elif label == 'Total Number of Stocks':
        return 'Equity characteristics-Total Number of Stocks' 
    else:
        return 'Unassigned'

In [122]:
chars_melted['eA Characteristic'] = chars_melted['Label'].apply(assign_chars_name)

In [123]:
chars_values = chars_melted['Label'].value_counts().index.to_list()

In [124]:
## Use 'Label' column to create 'Category' column, this is again, only for market cap

def assign_chars_category(label):
    if label in chars_values:
        return 'Equity characteristics'
    else:
        return 'Houston, we have a prob'

In [125]:
chars_melted['Category'] = chars_melted['Label'].apply(assign_chars_category)

In [126]:
## Rearrange order of cols
chars_melted = chars_melted[['Date','Product', 'Category', 'Label', 'Value', 'eA Characteristic']]

In [127]:
## Run function to change eVestment Product name
chars_melted['Product'] = chars_melted['Product'].apply(change_name)
chars_melted.head(3)

,Date,Product,Category,Label,Value,eA Characteristic
0,3/31/2021,ABC,Equity characteristics,Total Number of Stocks,422,Equity characteristics-Total Number of Stocks
1,3/31/2021,ABC,Equity characteristics,Price/Earnings Ratio (1 Year Trailing),34.44,Equity characteristics-Price/Earnings Ratio (1...
2,3/31/2021,ABC,Equity characteristics,P/E Ratio (1Yr Forward),24.35,Equity characteristics-P/E Ratio (1Yr Forward)


#### Step 9 - Import `country` Tab

In [128]:
country_df = pd.read_excel('eA_template.xlsx', sheet_name = 'country', skiprows=5, skipfooter =2) 
country_headers = country_df.iloc[0]
country_df = pd.DataFrame(country_df.values[2:], columns = country_headers)
country_df.columns.values[0] = 'Country'

#### Step 10 - Transform `country` data into Seismic format

In [129]:
# Create list of all products included in sector tab.  Will provide as variable below.  This is scalable, if add new products.
country_cols = list(country_df.columns)
# Excludes Label and MSCI ACWI from list
country_cols = country_cols[1:]
country_cols

['ABC', 'XYX', 'ZZZ']

In [130]:
#  Repeat Sector buckets and create Product Name column for each product
country_melted = pd.melt(country_df, id_vars=['Country'], value_vars = country_cols, var_name = 'Product', value_name = 'Value')

In [131]:
# Replace  -- from import file with 0
country_melted['Value'].replace('--', 0, inplace = True)

In [132]:
## Add Date Column
country_melted.insert(0, 'Date', report_date)

In [133]:
## Run function to change eVestment Product name
country_melted['Product'] = country_melted['Product'].apply(change_name)

In [134]:
## Rearrange order of cols
country_melted = country_melted[['Date','Product', 'Country', 'Value']]

In [135]:
country_melted.pivot_table(index = 'Product', values = 'Value', aggfunc = sum)

,Value
Product,
ABC,100.0
XYX,100.0
ZZZ,100.0


#### Step 11 - Create final dataframe named `final_df`

In [136]:
final_df = pd.concat([mc_melted, sector_melted, cash_melted, chars_melted])

In [137]:
final_df

,Date,Product,Category,Label,Value,eA Characteristic
0,3/31/2021,ABC,Capitalization Ranges,> $50 Billion,43.9,Capitalization Ranges-> $50 Billion
1,3/31/2021,ABC,Capitalization Ranges,$25-50 Billion,32.46,Capitalization Ranges-$25-50 Billion
2,3/31/2021,ABC,Capitalization Ranges,$15-25 Billion,11.972483,Capitalization Ranges-$15-25 Billion
3,3/31/2021,ABC,Capitalization Ranges,$10-15 Billion,7.205876,Capitalization Ranges-$10-15 Billion
4,3/31/2021,ABC,Capitalization Ranges,$7.5-10 Billion,3.131328,Capitalization Ranges-$7.5-10 Billion
...,...,...,...,...,...,...
52,3/31/2021,ZZZ,Equity characteristics,Price/Sales,3.768427,Equity characteristics-Price Sales
53,3/31/2021,ZZZ,Equity characteristics,Total Debt/Total Equity,71.910238,Equity characteristics-Total Debt/Total Equity
54,3/31/2021,ZZZ,Equity characteristics,Weighted Average Market Cap ($ millions),47921.023438,Equity characteristics-Weighted Average Market...
55,3/31/2021,ZZZ,Equity characteristics,Median Market Cap,8633.395508,Equity characteristics-Median Market Cap


#### Step 12 - Create excel output file with 2 tabs

In [138]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('eA_output.xlsx', engine = 'xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
final_df.to_excel(writer, sheet_name='Characteristics', index=False)
country_melted.to_excel(writer, sheet_name = "Countries without Cash", index=False)

# Get the xlsxwriter objects from the dataframe writer object.
workbook = writer.book
worksheet_1 = writer.sheets['Characteristics']
worksheet_2 = writer.sheets['Countries without Cash']

# Add some cell formats.
format1 = workbook.add_format()
format2 = workbook.add_format()

worksheet_1.set_column('A:A', 13, format1)
worksheet_1.set_column('B:B', 66, format1)
worksheet_1.set_column('C:E', 26, format1)
worksheet_1.set_column('F:F', 45, format1)

worksheet_2.set_column('A:A', 13, format2)
worksheet_2.set_column('B:B', 66, format2)
worksheet_2.set_column('C:C', 25, format2)
worksheet_2.set_column('D:D', 12, format2)

writer.save()